## Loading Data

To start using the package functionalities, we need to load the data first. The data can be provided as a CSV file, which should include the following columns:

<br>
<center>

| Column Name | Data Type | Description                  |
|-------------|-----------|------------------------------|
| isbn        | str       | the ISBN of the book         |
| title       | str       | the title of the book        |
| description | str       | the description of the book  |
| author      | list[str] | the author(s) of the book    |
| genre       | list[str] | the genre(s) of the book     |
| available   | bool      | the availability of the book |

</center>
<br>

<i>Note: The table names might be different than the ones mentioned above. In that case, an additional parameter should be provided to specify the column names (`column_names` in `process_data()`).</i>

Before loading the data we need to process it. The `process_data()` function will process the data and generate the embeddings (this might take a while). It will split the data into parts, generate embeddings, store them in a specified folder. The folder will have numbered CSVs with the processed data and PKLs with the embeddings.

In [ ]:
from kitab.utils import process_data

filepath = "data.csv"
destination_folder = "data"

process_data(filepath, destination_folder)

Then we need to load the data into the database. For this, we need to provide the following database connection details in the `.env`:

<br>
<center>

| Parameter   | Description |
|-------------|-------------|
| DB_USER     | str         |
| DB_PASSWORD | str         |
| DB_HOST     | str         |
| DB_PORT     | str or int  |
| DB_NAME     | str         |

</center>
<br>

After that, we can load the data into the database using the `load_data()` function.

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from kitab.db.get_data import load_data

load_data("data")

Now, we're ready to use the package functionalities.

## Model

The package includes a model that can be used to recommend books. The model computes similarity search between book descriptions by using the embeddings to find the most similar books to the given book. The model can be used to recommend books based on the book description, ISBN, or title. For the latter two, the book must be present in the database, so that the description can be retrieved and used for the similarity search. Let's have a look at these three different ways to recommend books.

In [ ]:
from kitab.recommendation_model.models import recommend_books

description = "In this thrilling detective tale, a group of childhood friends accidentally stumble upon an ancient artifact hidden in their clubhouse. Little do they know, their discovery thrusts them into a dangerous conspiracy spanning centuries. As they uncover clues, they race against time to prevent a cataclysmic event that could reshape the world. Join them on a heart-pounding journey through shadows and secrets in this gripping mystery."

recommend_books(description, n=5)

In [ ]:
from kitab.recommendation_model.models import recommend_books_by_ISBN

recommend_books_by_ISBN(ISBN="1442942355", n=5)

In [ ]:
from kitab.recommendation_model.models import recommend_books_by_title

recommend_books_by_title(title="The Ghostly Rental", n=5)

## Database Functions

We have implemented a range of functions to interact with the database. These functions can be used to retrieve books (by ISBN or title); add or update books; get lists of authors and genres for specific books (by ISBNs); add and retrieve recommendation logs; and get a certain table. Let's test these functions one by one.

In [ ]:
from kitab.db.functions import get_book_by_ISBN

get_book_by_ISBN("1442942355")

In [ ]:
from kitab.db.functions import get_book_by_title

get_book_by_title("The Ghostly Rental")

In [ ]:
from kitab.db.functions import add_book_db

add_book_db({
    "ISBN": "123456789",
    "title": "Thrilling Detective Tale",
    "description": "In this thrilling detective tale, a group of childhood friends accidentally stumble upon an ancient artifact hidden in their clubhouse. Little do they know, their discovery thrusts them into a dangerous conspiracy spanning centuries. As they uncover",
    "available": True,
    "authors": ["Arthur Conan Doyle", "Agatha Christie"], 
    "genres": ["Detective", "Fiction"]
})

In [ ]:
from kitab.db.functions import update_book_db

update_book_db("1442942355", {
    "available": True,
    "genres": ["Horror", "Short Stories", "Mystery"]
})

In [ ]:
from kitab.db.functions import get_authors

get_authors(ISBNs=["1442942355", "1613720211"])

In [ ]:
from kitab.db.functions import get_genres

get_genres(ISBNs=["1442942355", "1613720211"])

In [ ]:
from kitab.db.functions import add_recommendation_log

add_recommendation_log(description="In a masterful blend of psychological intrigue and spectral disturbances, this novel unfurls the complex life of Clara. Her internal struggles are mirrored by eerie, inexplicable occurrences, weaving a tale that is both deeply personal and chillingly atmospheric, offering an unparalleled exploration of the human psyche shadowed by the paranormal.", recommendation_isbn="1442942355", successful=True)

In [ ]:
from kitab.db.functions import get_history_by_recommendation_isbn

get_history_by_recommendation_isbn(recommendation_isbn="1442942355")

In [ ]:
from kitab.db.functions import get_table_from_db

get_table_from_db("book", conditions={"available": True})